In [1]:
from pathlib import Path
from collections import defaultdict
import mat_ceng as mc


In [2]:
folder_name = Path('.').resolve()
file_name = folder_name / 'sample_warning.txt'

result = {
    'total_warnings': 0,
    'colleced_warnings': 0,
    'warnings': defaultdict(list),  # Using defaultdict for automatic list initialization,
    'warning_locations': []
}

unit_conversion = 1000.0

# Read file line by line
with open(file_name, 'r') as file:
    for indx, line in enumerate(file):
        if indx == 0:
            result['total_warnings'] = int(line.strip().split(', ')[1].split(' ')[0])
            continue
        row = line.strip()
        if 'Check at (' in row:
            result['colleced_warnings'] += 1
            coord = row.split('Check at (')[1].split(')')[0].split(' ')
            coord = [float(x) * unit_conversion for x in coord]
            coord = tuple(coord)
            result['warning_locations'].append(coord)
            result['warnings'][coord].append(row)

result['warnings'] = dict(result['warnings'])  # Convert defaultdict to dict
result['warning_locations'] = list(set(result['warning_locations']))  # Remove duplicates
print(f'total warnings: {result["total_warnings"]}, collected warnings: {result["colleced_warnings"]}')


total warnings: 43, collected warnings: 43


In [9]:
mc.csi.CsiHelper.connect_to_etabs(units= mc.csi.etabs_units['N_mm_C'])
mc.csi.CsiHelper.set_etabs_units()
mc.csi.CsiHelper.refresh_view()

In [10]:
etabs_groups = {
    'NumberNames': 0,
    'MyName': []
}

etabs_groups_names = mc.csi.get_etabs_groups()
etabs_groups_names


['All', 'Warnings']

In [11]:

warning_group_name = 'Warnings'

if warning_group_name not in etabs_groups_names:
    mc.csi.create_etabs_group(warning_group_name)


In [12]:

etabs_added_areas = [mc.csi.add_area(x,warning_group_name) for x in result['warning_locations']]
etabs_added_areas 


['27',
 '32',
 '47',
 '98',
 '100',
 '110',
 '111',
 '121',
 '124',
 '126',
 '127',
 '128',
 '129',
 '130',
 '132',
 '133',
 '134',
 '135',
 '136']

In [13]:
import pandas as pd

df = pd.DataFrame(result['warnings'].items(), columns=['Coordinates', 'Warnings'])
df['area_name'] = etabs_added_areas
df['Warnings'] = df['Warnings'].apply(lambda x: '\n'.join(x))
df.set_index('area_name', inplace=True)
df.to_excel('warnings.xlsx', index=True)
df.head(5)

,Coordinates,Warnings
area_name,,
27,"(1904.0, -61728.0, 16500.0)","UPPER ROOF FLOOR PLAN, B420 & B307 are too clo..."
32,"(1953.0, -61730.0, 6900.0)","FIRST FRAMING FLOOR PLAN, B386 & B559 are too ..."
47,"(-1780.0, -61641.0, 6900.0)","FIRST FRAMING FLOOR PLAN, F9 & B392 are too cl..."
98,"(-2252.0, -61629.0, 6900.0)","FIRST FRAMING FLOOR PLAN, 878 & 888 are too cl..."
100,"(-2255.0, -61621.0, 6900.0)","FIRST FRAMING FLOOR PLAN, 888 & B387 are too c..."


In [14]:
mc.csi.CsiHelper.release_csi_models()